<a href="https://colab.research.google.com/github/JacobDowns/glamacles_test/blob/main/train_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install nvidia-modulus nvidia-modulus-sym
! git clone https://github.com/JacobDowns/glamacles_test
! cd glamacles_test

  Using cached nvidia_modulus-0.6.0-py3-none-any.whl (320 kB)
  Using cached nvidia_modulus.sym-1.5.0-py3-none-any.whl (291 kB)
  Using cached s3fs-2024.5.0-py3-none-any.whl (29 kB)
  Using cached timm-0.5.4-py3-none-any.whl (431 kB)
  Using cached torch_optimizer-0.3.0-py3-none-any.whl (61 kB)
  Using cached notebook-6.4.12-py3-none-any.whl (9.9 MB)
  Using cached pytorch_ranger-0.1.1-py3-none-any.whl (14 kB)
INFO: pip is looking at multiple versions of nbconvert to determine which version is compatible with other requirements. This could take a while.
  Using cached nbconvert-7.16.4-py3-none-any.whl (257 kB)
  Attempting uninstall: nbconvert
    Found existing installation: nbconvert 6.5.4
    Uninstalling nbconvert-6.5.4:
      Successfully uninstalled nbconvert-6.5.4
  Attempting uninstall: notebook
    Found existing installation: notebook 6.5.5
    Uninstalling notebook-6.5.5:
      Successfully uninstalled notebook-6.5.5
ERROR: pip's dependency resolver does not currently take i

In [ ]:
import os
arch = os.getenv("ARGS", "real")

try:
    import google.colab  # noqa: F401
except ImportError:
    import firedrake as fd
else:
    try:
        import firedrake as fd
    except ImportError:
        if arch != "complex":
            !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
        else:
            !wget "https://fem-on-colab.github.io/releases/firedrake-install-complex.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
        import firedrake as fd

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from model import SpecFO
import numpy as np
import matplotlib.pyplot as plt
data = np.load('/content/drive/MyDrive/glamacles/data.npy')
model = SpecFO()

/usr/local/lib/python3.10/dist-packages/firedrake/interpolation.py:385: FutureWarning: The use of `interpolate` to perform the numerical interpolation is deprecated.
This feature will be removed very shortly.

Instead, import `interpolate` from the `firedrake.__future__` module to update
the interpolation's behaviour to return the symbolic `ufl.Interpolate` object associated
with this interpolation.

You can then assemble the resulting object to get the interpolated quantity
of interest. For example,

```
from firedrake.__future__ import interpolate
...

assemble(interpolate(expr, V))
```

Alternatively, you can also perform other symbolic operations on the interpolation operator, such as taking
the derivative, and then assemble the resulting form.

  warnings.warn("""The use of `interpolate` to perform the numerical interpolation is deprecated.


In [ ]:
import torch
from modulus.models.fno import FNO
from torch.utils.data import TensorDataset, DataLoader
from torch.nn import MSELoss

fno = FNO(
    in_channels=2,
    out_channels=4,
    decoder_layers=1,
    decoder_layer_size=32,
    dimension=2,
    latent_channels=32,
    num_fno_layers=4,
    num_fno_modes=16,
    padding=9,
).cuda()

optimizer = torch.optim.Adam(fno.parameters(), lr=1e-4)

In [ ]:
data = torch.tensor(data, dtype=torch.float32)
data.shape

<ipython-input-110-67482221ac71>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data, dtype=torch.float32)


torch.Size([2500, 6, 101, 101])

In [ ]:
# Model inputs B and H
X = data[:,[0,1],:,:]
# MOdel outputs (velocity)
Y = data[:,2:,:,:]

train_dataset = TensorDataset(X[0:2000], Y[0:2000])
validation_dataset = TensorDataset(X[2000:], Y[2000:])
#train_data = dataset[0:2000]
#validation_data = dataset[2000:]

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=1, shuffle=True)

# Number of epochs
epochs = 100

# Loss function
mse = MSELoss()

In [ ]:
# Training loop

for epoch in range(epochs):

  fno.train()
  loss_avg = torch.tensor(0.)
  num = 0
  for i,(x,y) in enumerate(train_loader):
    optimizer.zero_grad()
    x = x.cuda()
    y = y.cuda()
    y_fno = fno(x)

    loss = mse(y_fno, y)
    loss_avg += loss.cpu()
    loss.backward()
    num += 1

  loss_avg /= num
  print('loss', loss_avg.item())


loss 7418972.0
loss 7418972.5
loss 7418973.5
